Create a pandas dataframe with election data from https://elections.sos.state.tx.us/elchist331_race832.htm

In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
import json

In [2]:
url = 'https://elections.sos.state.tx.us/elchist331_race832.htm'
url

'https://elections.sos.state.tx.us/elchist331_race832.htm'

### Load the webpage

In [3]:
webpage_request = r.get(url)
webpage_request

<Response [200]>

In [4]:
# create the soup object
soup = bs(webpage_request.content)
soup

<html>
<body background="tempct.gif" bgproperties="fixed">
<h2 align="center">Office of the Secretary of State</h2>
<br/>
<br/>
<form method="post">
<h2 align="center">2018 General Election</h2><h3 align="center">11/6/2018</h3>U. S. Senator - <table border="1"><tr><th colspan="1">...</th><th colspan="1">Ted Cruz</th><th colspan="1">Beto O'Rourke</th><th colspan="1">Neal M. Dikeman</th><th colspan="1">...</th><th colspan="1">...</th><th colspan="1">...</th></tr>
<tr><th colspan="1">...</th><th colspan="1"></th><th colspan="1"></th><th colspan="1"></th><th colspan="1">Total</th><th colspan="1">Total</th><th colspan="1">...</th></tr>
<tr><th>County</th><th colspan="1">REP</th><th colspan="1">DEM</th><th colspan="1">LIB</th><th>Votes</th><th>Voters</th><th>TurnOut</th></tr>
<tr align="right"><td align="left">ALL COUNTIES</td><td>4,260,553</td><td>4,045,632</td><td>65,470</td><td>8,371,655</td><td>15,793,257</td><td>53.01%</td></tr>
<tr align="right"><td align="left"><font color="teal">ANDE

we want to grab all the `<tr>` tags from the table from this page

In [5]:
table = soup.select('table tr')
len(table)

258

In [9]:
table[2]

<tr><th>County</th><th colspan="1">REP</th><th colspan="1">DEM</th><th colspan="1">LIB</th><th>Votes</th><th>Voters</th><th>TurnOut</th></tr>

In [10]:
# create the columns of the table
columns = [x.get_text() for x in table[2].find_all('th')]
columns

['County', 'REP', 'DEM', 'LIB', 'Votes', 'Voters', 'TurnOut']

In [11]:
[x for x in table if x.find_all('td')]

[<tr align="right"><td align="left">ALL COUNTIES</td><td>4,260,553</td><td>4,045,632</td><td>65,470</td><td>8,371,655</td><td>15,793,257</td><td>53.01%</td></tr>,
 <tr align="right"><td align="left"><font color="teal">ANDERSON</font></td><td><font color="maroon">11,335</font></td><td><font color="maroon">3,307</font></td><td><font color="maroon">94</font></td><td><font color="teal">14,736</font></td><td><font color="teal">28,487</font></td><td><font color="teal">51.73%</font></td></tr>,
 <tr align="right"><td align="left"><font color="teal">ANDREWS</font></td><td><font color="maroon">3,338</font></td><td><font color="maroon">776</font></td><td><font color="maroon">17</font></td><td><font color="teal">4,131</font></td><td><font color="teal">9,574</font></td><td><font color="teal">43.15%</font></td></tr>,
 <tr align="right"><td align="left"><font color="teal">ANGELINA</font></td><td><font color="maroon">19,166</font></td><td><font color="maroon">7,130</font></td><td><font color="maroon">

In [12]:
[[td for td in x.find_all('td')] for x in table if x.find_all('td')]

[[<td align="left">ALL COUNTIES</td>,
  <td>4,260,553</td>,
  <td>4,045,632</td>,
  <td>65,470</td>,
  <td>8,371,655</td>,
  <td>15,793,257</td>,
  <td>53.01%</td>],
 [<td align="left"><font color="teal">ANDERSON</font></td>,
  <td><font color="maroon">11,335</font></td>,
  <td><font color="maroon">3,307</font></td>,
  <td><font color="maroon">94</font></td>,
  <td><font color="teal">14,736</font></td>,
  <td><font color="teal">28,487</font></td>,
  <td><font color="teal">51.73%</font></td>],
 [<td align="left"><font color="teal">ANDREWS</font></td>,
  <td><font color="maroon">3,338</font></td>,
  <td><font color="maroon">776</font></td>,
  <td><font color="maroon">17</font></td>,
  <td><font color="teal">4,131</font></td>,
  <td><font color="teal">9,574</font></td>,
  <td><font color="teal">43.15%</font></td>],
 [<td align="left"><font color="teal">ANGELINA</font></td>,
  <td><font color="maroon">19,166</font></td>,
  <td><font color="maroon">7,130</font></td>,
  <td><font color="maro

In [13]:
table_rows = [[td.get_text() for td in x.find_all('td')] for x in table if x.find_all('td')]
table_rows

[['ALL COUNTIES',
  '4,260,553',
  '4,045,632',
  '65,470',
  '8,371,655',
  '15,793,257',
  '53.01%'],
 ['ANDERSON', '11,335', '3,307', '94', '14,736', '28,487', '51.73%'],
 ['ANDREWS', '3,338', '776', '17', '4,131', '9,574', '43.15%'],
 ['ANGELINA', '19,166', '7,130', '153', '26,449', '51,751', '51.11%'],
 ['ARANSAS', '6,677', '2,247', '56', '8,980', '17,308', '51.88%'],
 ['ARCHER', '3,208', '376', '18', '3,602', '6,317', '57.02%'],
 ['ARMSTRONG', '819', '74', '6', '899', '1,428', '62.96%'],
 ['ATASCOSA', '7,753', '4,332', '119', '12,204', '27,338', '44.64%'],
 ['AUSTIN', '8,722', '2,241', '84', '11,047', '19,406', '56.93%'],
 ['BAILEY', '1,204', '405', '7', '1,616', '3,727', '43.36%'],
 ['BANDERA', '7,643', '1,865', '76', '9,584', '15,869', '60.39%'],
 ['BASTROP', '15,067', '12,082', '312', '27,461', '47,438', '57.89%'],
 ['BAYLOR', '1,070', '156', '9', '1,235', '2,365', '52.22%'],
 ['BEE', '4,342', '2,811', '64', '7,217', '15,883', '45.44%'],
 ['BELL', '47,437', '38,417', '723', '8

In [14]:
county_dict = {v:"" for v in columns}
county_dict

{'County': '',
 'REP': '',
 'DEM': '',
 'LIB': '',
 'Votes': '',
 'Voters': '',
 'TurnOut': ''}

In [15]:
[(a,b) for a,b in zip(table_rows[0],columns)]

[('ALL COUNTIES', 'County'),
 ('4,260,553', 'REP'),
 ('4,045,632', 'DEM'),
 ('65,470', 'LIB'),
 ('8,371,655', 'Votes'),
 ('15,793,257', 'Voters'),
 ('53.01%', 'TurnOut')]

In [16]:
d = [{i:x for i,x in zip(columns,a)} for a in table_rows]
d

[{'County': 'ALL COUNTIES',
  'REP': '4,260,553',
  'DEM': '4,045,632',
  'LIB': '65,470',
  'Votes': '8,371,655',
  'Voters': '15,793,257',
  'TurnOut': '53.01%'},
 {'County': 'ANDERSON',
  'REP': '11,335',
  'DEM': '3,307',
  'LIB': '94',
  'Votes': '14,736',
  'Voters': '28,487',
  'TurnOut': '51.73%'},
 {'County': 'ANDREWS',
  'REP': '3,338',
  'DEM': '776',
  'LIB': '17',
  'Votes': '4,131',
  'Voters': '9,574',
  'TurnOut': '43.15%'},
 {'County': 'ANGELINA',
  'REP': '19,166',
  'DEM': '7,130',
  'LIB': '153',
  'Votes': '26,449',
  'Voters': '51,751',
  'TurnOut': '51.11%'},
 {'County': 'ARANSAS',
  'REP': '6,677',
  'DEM': '2,247',
  'LIB': '56',
  'Votes': '8,980',
  'Voters': '17,308',
  'TurnOut': '51.88%'},
 {'County': 'ARCHER',
  'REP': '3,208',
  'DEM': '376',
  'LIB': '18',
  'Votes': '3,602',
  'Voters': '6,317',
  'TurnOut': '57.02%'},
 {'County': 'ARMSTRONG',
  'REP': '819',
  'DEM': '74',
  'LIB': '6',
  'Votes': '899',
  'Voters': '1,428',
  'TurnOut': '62.96%'},
 {

In [20]:
# We want to don't want the numeric values to be in strings (' ').
# To change this we want to replace the commas(',') with nothing ("")

int_keys = ['REP', 'DEM','LIB', 'Votes', 'Voters']
int_keys

['REP', 'DEM', 'LIB', 'Votes', 'Voters']

In [21]:
new_list = [{k:(int(v.replace(",","")) if k in int_keys else v) for k,v in x.items()} for x in d]
new_list

[{'County': 'ALL COUNTIES',
  'REP': 4260553,
  'DEM': 4045632,
  'LIB': 65470,
  'Votes': 8371655,
  'Voters': 15793257,
  'TurnOut': '53.01%'},
 {'County': 'ANDERSON',
  'REP': 11335,
  'DEM': 3307,
  'LIB': 94,
  'Votes': 14736,
  'Voters': 28487,
  'TurnOut': '51.73%'},
 {'County': 'ANDREWS',
  'REP': 3338,
  'DEM': 776,
  'LIB': 17,
  'Votes': 4131,
  'Voters': 9574,
  'TurnOut': '43.15%'},
 {'County': 'ANGELINA',
  'REP': 19166,
  'DEM': 7130,
  'LIB': 153,
  'Votes': 26449,
  'Voters': 51751,
  'TurnOut': '51.11%'},
 {'County': 'ARANSAS',
  'REP': 6677,
  'DEM': 2247,
  'LIB': 56,
  'Votes': 8980,
  'Voters': 17308,
  'TurnOut': '51.88%'},
 {'County': 'ARCHER',
  'REP': 3208,
  'DEM': 376,
  'LIB': 18,
  'Votes': 3602,
  'Voters': 6317,
  'TurnOut': '57.02%'},
 {'County': 'ARMSTRONG',
  'REP': 819,
  'DEM': 74,
  'LIB': 6,
  'Votes': 899,
  'Voters': 1428,
  'TurnOut': '62.96%'},
 {'County': 'ATASCOSA',
  'REP': 7753,
  'DEM': 4332,
  'LIB': 119,
  'Votes': 12204,
  'Voters': 27

In [22]:
# OLD IDEA: to get counties added to the list. UPDATE ABOVE: used if/else statement in dictionary comprehension
# new_list = []
# for x in d:
#     for k in x:
#         if k in int_keys:
#             x[k]=int(x[k].replace(",",""))
#     new_list.append(x)

# new_list

In [23]:
# Update Turnout field to numeric. Replace '.' and '%'. Divide by 100 to get a float percent. EX (10% = .10)

new_list = [{k:(float(v.replace("%","")) if k=='TurnOut' else v) for k,v in x.items()} for x in new_list]
new_list

[{'County': 'ALL COUNTIES',
  'REP': 4260553,
  'DEM': 4045632,
  'LIB': 65470,
  'Votes': 8371655,
  'Voters': 15793257,
  'TurnOut': 53.01},
 {'County': 'ANDERSON',
  'REP': 11335,
  'DEM': 3307,
  'LIB': 94,
  'Votes': 14736,
  'Voters': 28487,
  'TurnOut': 51.73},
 {'County': 'ANDREWS',
  'REP': 3338,
  'DEM': 776,
  'LIB': 17,
  'Votes': 4131,
  'Voters': 9574,
  'TurnOut': 43.15},
 {'County': 'ANGELINA',
  'REP': 19166,
  'DEM': 7130,
  'LIB': 153,
  'Votes': 26449,
  'Voters': 51751,
  'TurnOut': 51.11},
 {'County': 'ARANSAS',
  'REP': 6677,
  'DEM': 2247,
  'LIB': 56,
  'Votes': 8980,
  'Voters': 17308,
  'TurnOut': 51.88},
 {'County': 'ARCHER',
  'REP': 3208,
  'DEM': 376,
  'LIB': 18,
  'Votes': 3602,
  'Voters': 6317,
  'TurnOut': 57.02},
 {'County': 'ARMSTRONG',
  'REP': 819,
  'DEM': 74,
  'LIB': 6,
  'Votes': 899,
  'Voters': 1428,
  'TurnOut': 62.96},
 {'County': 'ATASCOSA',
  'REP': 7753,
  'DEM': 4332,
  'LIB': 119,
  'Votes': 12204,
  'Voters': 27338,
  'TurnOut': 44.

In [24]:
pd.DataFrame(new_list).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 7 columns):
County     255 non-null object
DEM        255 non-null int64
LIB        255 non-null int64
REP        255 non-null int64
TurnOut    255 non-null float64
Voters     255 non-null int64
Votes      255 non-null int64
dtypes: float64(1), int64(5), object(1)
memory usage: 14.0+ KB


In [25]:
pd.DataFrame(new_list)

,County,DEM,LIB,REP,TurnOut,Voters,Votes
0,ALL COUNTIES,4045632,65470,4260553,53.01,15793257,8371655
1,ANDERSON,3307,94,11335,51.73,28487,14736
2,ANDREWS,776,17,3338,43.15,9574,4131
3,ANGELINA,7130,153,19166,51.11,51751,26449
4,ARANSAS,2247,56,6677,51.88,17308,8980
5,ARCHER,376,18,3208,57.02,6317,3602
6,ARMSTRONG,74,6,819,62.96,1428,899
7,ATASCOSA,4332,119,7753,44.64,27338,12204
8,AUSTIN,2241,84,8722,56.93,19406,11047
9,BAILEY,405,7,1204,43.36,3727,1616
